In [ ]:
# Copyright 2024 Forus(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image Recognition


### Install Vertex AI SDK for Python


In [1]:
!pip3 install --upgrade --quiet google-cloud-aiplatform \
                                google-cloud-vision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.9/486.9 kB 23.1 MB/s eta 0:00:00


### Authentication to access to the GCP

In [2]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik

Updated property [core/project].


In [3]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization


In [4]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    Tool
)

import vertexai
from vertexai.preview.generative_models import grounding

vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(MODEL_NAME)

## Detect Web entities and pages by Vision API
* https://cloud.google.com/vision/docs/detecting-web

### Helper functions

In [5]:

def web_detection(path:str):
  """ Detects web annotations given an image. """

  from google.cloud import vision

  client = vision.ImageAnnotatorClient() # ImageAnnotatorClient 클래스의 인스턴스 생성

  with open(path, "rb") as image_file:
      content = image_file.read()

  image = vision.Image(content=content)

  #[web_detection_requests] https://cloud.google.com/vision/docs/detecting-web#web_detection_requests
  response = client.web_detection(image=image) # 인스턴스를 통해 web_detection() 메서드 호출
  web_detection = response.web_detection

  return web_detection, web_detection.best_guess_labels[0].label

def search(prompt:str)->str:

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  response = model.generate_content(
      prompt,
      tools=[tool],
      generation_config=GenerationConfig(
          temperature=0.0,
      ),
  )

  return response


def print_grounding_response(response: GenerationResponse):

    """Prints Gemini response with grounding citations."""
    grounding_metadata = response.candidates[0].grounding_metadata

    # Citation indices are in byte units
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)

    prev_index = 0
    markdown_text = ""

    for grounding_support in grounding_metadata.grounding_supports:
        text_segment = text_bytes[
            prev_index : grounding_support.segment.end_index
        ].decode(ENCODING)

        footnotes_text = ""
        for grounding_chunk_index in grounding_support.grounding_chunk_indices:
            footnotes_text += f"[{grounding_chunk_index + 1}]"

        markdown_text += f"{text_segment} {footnotes_text}\n"
        prev_index = grounding_support.segment.end_index

    if prev_index < len(text_bytes):
        markdown_text += str(text_bytes[prev_index:], encoding=ENCODING)

    markdown_text += "\n----\n## Grounding Sources\n"

    if grounding_metadata.web_search_queries:
        markdown_text += (
            f"\n**Web Search Queries:** {grounding_metadata.web_search_queries}\n"
        )
        if grounding_metadata.search_entry_point:
            markdown_text += f"\n**Search Entry Point:**\n {grounding_metadata.search_entry_point.rendered_content}\n"
    elif grounding_metadata.retrieval_queries:
        markdown_text += (
            f"\n**Retrieval Queries:** {grounding_metadata.retrieval_queries}\n"
        )

    markdown_text += "### Grounding Chunks\n"

    for index, grounding_chunk in enumerate(
        grounding_metadata.grounding_chunks, start=1
    ):
        context = grounding_chunk.web or grounding_chunk.retrieved_context
        if not context:
            print(f"Skipping Grounding Chunk {grounding_chunk}")
            continue

        markdown_text += f"{index}. [{context.title}]({context.uri})\n"
        markdown_text += f"\n\n- {context.uri}\n\n"

    # display(Markdown(markdown_text))
    return Markdown(markdown_text)

### Image Upload


In [8]:
from google.colab import files
from IPython.display import Image
uploaded = files.upload()


Saving dmz.jpg to dmz.jpg


In [11]:
detection_results, best_guess_label = web_detection("./cha.jpg")

top_10 = []
print(f"Best Guess Label : {best_guess_label}")
print( "-"*100)
for entity in detection_results.web_entities:
    top_10.append(f"{entity.description}")
    # top_10.append(f"{entity.description}, Score : {entity.score}")

print(top_10[:5])

print( "-"*100)

detection_results

Best Guess Label : gentleman
----------------------------------------------------------------------------------------------------
['Cha Seung-won', 'Three Meals a Day: Light', 'Actor', 'tvN', 'K-pop']
----------------------------------------------------------------------------------------------------


web_entities {
  entity_id: "/m/0k_n3g"
  score: 14.2725
  description: "Cha Seung-won"
}
web_entities {
  entity_id: "/g/11w95l2xfd"
  score: 1.0605
  description: "Three Meals a Day: Light"
}
web_entities {
  entity_id: "/m/02hrh1q"
  score: 0.6615
  description: "Actor"
}
web_entities {
  entity_id: "/m/0j9q4b5"
  score: 0.5125
  description: "tvN"
}
web_entities {
  entity_id: "/m/02yh8l"
  score: 0.3983
  description: "K-pop"
}
web_entities {
  entity_id: "/g/11byzf3gtr"
  score: 0.3288
  description: "BLACKPINK"
}
web_entities {
  entity_id: "/g/11pkltmnjd"
  score: 0.324060023
  description: "Crest 72"
}
web_entities {
  entity_id: "/g/11wfptbnxj"
  score: 0.2688
  description: "Mantra (Preview)"
}
web_entities {
  entity_id: "/m/06qd3"
  score: 0.117120005
  description: "South Korea"
}
web_entities {
  entity_id: "/m/0g41cwq"
  score: 0.07824
  description: "Yoo Hae-jin"
}
partial_matching_images {
  url: "https://blog.kakaocdn.net/dn/bFOW7a/btsJKh43S32/Sa9I9b8KPQwCTkkn9ls9j1/

### Search information from metadata

In [12]:
prompt = f"""
  당신은 주어진 참고정보를 바탕으로 정보를 자세하게 정리해주는 AI 어시스턴트 입니다.
  1. 인물에 초점을 두고 정리해주세요.
  2. 정리는 가급적 bullet point로 자세하게 정리해주세요.
  3. 한국어로 답변해주세요.

  참고정보 : {' '.join(top_10[:1])}

"""
response = search(prompt)
print_grounding_response(response)

## 차승원에 대한 정보 정리

**개인 정보**

* **본명:** 차승원
* **출생:** 1970년 6월 7일 (서울, 대한민국) [1]

* **나이:** 54세 (2024년 11월 28일 기준)
* **키:** 188cm [1]

* **배우자:** 이수진 (결혼: 1992년) [1]

* **자녀:** 차예니 (딸, 2003년생) [1]

* **소속사:** YG 엔터테인먼트

**경력**

* **모델 데뷔:** 1988년 [1]

* **배우 데뷔:** 1997년 영화 "홀리데이 인 서울" [1]

* **주요 활동:** 드라마, 영화, 예능 프로그램 [2]


**주요 작품**

* **영화:**
    * "리베라 메" (2000) - 방화범 역 [1]

    * "킥 더 문" (2001) - 큰 성공을 거둔 코미디 영화 [1]

    * "Jail Breakers" (2002) [1]

    * "마이 티처, 미스터 김" (2003) [1]

    * "고스트 하우스" (2004) [1]

    * "혈의 바다" (2005) - 시대극 스릴러
    * "내 아들" (2007) - 멜로 드라마 [1]

    * "독전" (2018)
    * "낙원의 밤" (2019) [1][2]

    * "싱크홀" (2021) - 재난 영화 [1][2]

    * "고산자, 대동여지도" (2023) - 김정호 역 (예정)
* **드라마:**
    * "뉴욕 스토리" (1995) - 시트콤
    * "보디가드" (2003)
    * "시티홀" (2009) [1]

    * "최고의 사랑" (2011) [3][1]

    * "아테나: 전쟁의 여신" (2010)
    * "화려한 유혹" (2015) [3]

    * "쓸쓸하고 찬란하神 - 도깨비" (2016) - 특별 출연
    * "화유기" (2017)
    * "하루" (2021) [1]

    * "우리들의 블루스" (2022) [1][2]

* **예능:**
    * "무한도전"
    * "삼시세끼"
    * "스페인 하숙"

**특징**

* 1990년대 초반부터 최정상급 모델로 활동하며 뛰어난 외모와 카리스마로 인기를 얻었음.
* 드라마, 영화, 예능 등 다양한 분야에서 활발하게 활동하며 폭넓은 연기 스펙트럼을 보여주는 배우임.
* 코미디, 드라마, 스릴러 등 다양한 장르를 넘나들며 뛰어난 연기력을 선보임.
* 모델 출신 배우로서 뛰어난 패션 감각과 스타일리시한 이미지를 가지고 있음.

**기타**

* 차승원은 모델로서 성공적인 커리어를 쌓은 후 배우로 전향하여 성공적인 배우 생활을 이어가고 있음. [1]

* 그는 다양한 작품을 통해 꾸준히 대중의 사랑을 받고 있으며, 한국 대표 배우 중 한 명으로 자리매김하고 있음.

----
## Grounding Sources

**Web Search Queries:** ['Cha Seung-won 영화 목록', 'Cha Seung-won 개요', 'Cha Seung-won 드라마 목록']

**Search Entry Point:**
 <style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webkit-tap-highlight-color: transparent;
}
.carousel {
  overflow: auto;
  scrollbar-width: none;
  white-space: nowrap;
  margin-right: -12px;
}
.headline {
  display: flex;
  margin-right: 4px;
}
.gradient-container {
  position: relative;
}
.gradient {
  position: absolute;
  transform: translate(3px, -9px);
  height: 36px;
  width: 9px;
}
@media (prefers-color-scheme: light) {
  .container {
    background-color: #fafafa;
    box-shadow: 0 0 0 1px #0000000f;
  }
  .headline-label {
    color: #1f1f1f;
  }
  .chip {
    background-color: #ffffff;
    border-color: #d2d2d2;
    color: #5e5e5e;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #f2f2f2;
  }
  .chip:focus {
    background-color: #f2f2f2;
  }
  .chip:active {
    background-color: #d8d8d8;
    border-color: #b6b6b6;
  }
  .logo-dark {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #fafafa 15%, #fafafa00 100%);
  }
}
@media (prefers-color-scheme: dark) {
  .container {
    background-color: #1f1f1f;
    box-shadow: 0 0 0 1px #ffffff26;
  }
  .headline-label {
    color: #fff;
  }
  .chip {
    background-color: #2c2c2c;
    border-color: #3c4043;
    color: #fff;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #353536;
  }
  .chip:focus {
    background-color: #353536;
  }
  .chip:active {
    background-color: #464849;
    border-color: #53575b;
  }
  .logo-light {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #1f1f1f 15%, #1f1f1f00 100%);
  }
}
</style>
<div class="container">
  <div class="headline">
    <svg class="logo-light" width="18" height="18" viewBox="9 9 35 35" fill="none" xmlns="http://www.w3.org/2000/svg">
      <path fill-rule="evenodd" clip-rule="evenodd" d="M42.8622 27.0064C42.8622 25.7839 42.7525 24.6084 42.5487 23.4799H26.3109V30.1568H35.5897C35.1821 32.3041 33.9596 34.1222 32.1258 35.3448V39.6864H37.7213C40.9814 36.677 42.8622 32.2571 42.8622 27.0064V27.0064Z" fill="#4285F4"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 43.8555C30.9659 43.8555 34.8687 42.3195 37.7213 39.6863L32.1258 35.3447C30.5898 36.3792 28.6306 37.0061 26.3109 37.0061C21.8282 37.0061 18.0195 33.9811 16.6559 29.906H10.9194V34.3573C13.7563 39.9841 19.5712 43.8555 26.3109 43.8555V43.8555Z" fill="#34A853"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M16.6559 29.8904C16.3111 28.8559 16.1074 27.7588 16.1074 26.6146C16.1074 25.4704 16.3111 24.3733 16.6559 23.3388V18.8875H10.9194C9.74388 21.2072 9.06992 23.8247 9.06992 26.6146C9.06992 29.4045 9.74388 32.022 10.9194 34.3417L15.3864 30.8621L16.6559 29.8904V29.8904Z" fill="#FBBC05"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 16.2386C28.85 16.2386 31.107 17.1164 32.9095 18.8091L37.8466 13.8719C34.853 11.082 30.9659 9.3736 26.3109 9.3736C19.5712 9.3736 13.7563 13.245 10.9194 18.8875L16.6559 23.3388C18.0195 19.2636 21.8282 16.2386 26.3109 16.2386V16.2386Z" fill="#EA4335"/>
    </svg>
    <svg class="logo-dark" width="18" height="18" viewBox="0 0 48 48" xmlns="http://www.w3.org/2000/svg">
      <circle cx="24" cy="23" fill="#FFF" r="22"/>
      <path d="M33.76 34.26c2.75-2.56 4.49-6.37 4.49-11.26 0-.89-.08-1.84-.29-3H24.01v5.99h8.03c-.4 2.02-1.5 3.56-3.07 4.56v.75l3.91 2.97h.88z" fill="#4285F4"/>
      <path d="M15.58 25.77A8.845 8.845 0 0 0 24 31.86c1.92 0 3.62-.46 4.97-1.31l4.79 3.71C31.14 36.7 27.65 38 24 38c-5.93 0-11.01-3.4-13.45-8.36l.17-1.01 4.06-2.85h.8z" fill="#34A853"/>
      <path d="M15.59 20.21a8.864 8.864 0 0 0 0 5.58l-5.03 3.86c-.98-2-1.53-4.25-1.53-6.64 0-2.39.55-4.64 1.53-6.64l1-.22 3.81 2.98.22 1.08z" fill="#FBBC05"/>
      <path d="M24 14.14c2.11 0 4.02.75 5.52 1.98l4.36-4.36C31.22 9.43 27.81 8 24 8c-5.93 0-11.01 3.4-13.45 8.36l5.03 3.85A8.86 8.86 0 0 1 24 14.14z" fill="#EA4335"/>
    </svg>
    <div class="gradient-container"><div class="gradient"></div></div>
  </div>
  <div class="carousel">
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfzKAu0ne14kvyucX4WaqJqGsyJKlfFriqjUhvvLXByXHqLZLN6mvzejTWjJWb6io4blYX9ak9zZWNIW8IDNMGyzB56xn5pYxRAZMlJABm2WROG8wa76Tts5sNbcaIC51RADEdaPkxgdKYG-_omAAeWNr-p_WlnLXhYy86kKPqXoCDUvnnT7dqw5qsLgZIjB-2fENFqE9L8oE8Z3ckWLR5nSnIcusUp99jXjA3B_Kfxvhys4Mi9VYb8=">Cha Seung-won 영화 목록</a>
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfyb054ICPSH8vhszRA1dy4mdEofj-Wbjbc1r34Pdw7OUmA4oQM7Aawv2NnBzDetGCWqTW1AIvhRsTV2B-teTiR6VoNpxVdpiJC0zKawp-iFIYlxKD2YTyy2sUZxG-bLe9UgInUYYOg-nk2Rai75CjEjkaPIPNk7l-sQbANE-DkSLhbh6yXvMYa_21t7St8OqVcVTdPSNcqp5_ZP3OCNLZkzVlFX">Cha Seung-won 개요</a>
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfxc9BfyZez8AihDByqOkCnB_uQK9HJpVFPTGo1SJLNsKy-JAUFnTdVRnliiqt66IORaTfIBA3r6Pemnwc-H4oVC1vYmm8fRjTY0J9XNlJSemubfMM44KB_TdjigWFU7PYs8ya2aurl_VhSC-kznvdVCqIp7VZsxJszK-TEdq6O8AGrO__i8BhLlBwKCDonJkxWAgovBYeiV7-URkcBdjLppp0Vgj-AYKFswcL7J-0b9OqmuVmGq4zEBBk9FlJyZnMc=">Cha Seung-won 드라마 목록</a>
  </div>
</div>

### Grounding Chunks
1. [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfzlzIfEwWNOEoOsAU7kTco4lcG9DXkFNxJPhvzxokRNmbAvUQ4akNkw3GII2pD7eBALCdtXkBWjN--CtR3KqDnRbP5d52LGcYWzSDmh0lv5FKtP_4GiNIWKNQWjtZSWZmrES-69tI4=)


- https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfzlzIfEwWNOEoOsAU7kTco4lcG9DXkFNxJPhvzxokRNmbAvUQ4akNkw3GII2pD7eBALCdtXkBWjN--CtR3KqDnRbP5d52LGcYWzSDmh0lv5FKtP_4GiNIWKNQWjtZSWZmrES-69tI4=

2. [ygfamily.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfx_tpX2erI7Tnx5gtNne12mC9Mlz75eZQpRAzWA-3BGgt9ovZvQ5qg9eXrhNuFWwFesNB9R9rb4PCq-y-zrE4M3iXDlCCqgFmLjOiU_-EG6CgysALSfjINPquNfX18VaENkpIJz97gYB5tlcQDPUg==)


- https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfx_tpX2erI7Tnx5gtNne12mC9Mlz75eZQpRAzWA-3BGgt9ovZvQ5qg9eXrhNuFWwFesNB9R9rb4PCq-y-zrE4M3iXDlCCqgFmLjOiU_-EG6CgysALSfjINPquNfX18VaENkpIJz97gYB5tlcQDPUg==

3. [imdb.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfwOB5DwyWD7ZwPFJGT-hW4tfk_kZ6zI-GB8CJZK_J-0gZDG9mbyrB093tOVAh49QcZcZLeNPtYoVdwMM1AZSneCUfZMFuLqzmIiI7xfuIgjK6RW6mjijm6THZNd73dQ4w==)


- https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfwOB5DwyWD7ZwPFJGT-hW4tfk_kZ6zI-GB8CJZK_J-0gZDG9mbyrB093tOVAh49QcZcZLeNPtYoVdwMM1AZSneCUfZMFuLqzmIiI7xfuIgjK6RW6mjijm6THZNd73dQ4w==



## Image recognition by Gemini

In [13]:
def detect_image(prompt:str,
                 image_uri:str,
                 response_schema)->str:
  """
  Function to call Gemini for Transcription and Translation.
  """

  from vertexai.generative_models import (
        Part,
        SafetySetting,
        GenerationConfig
  )
  import base64

  # Configuration to contorl LLM generation.
  generation_config = GenerationConfig(
      max_output_tokens = 8192,
      temperature=1,
      top_p =0.95,
      response_mime_type="application/json",
      response_schema=response_schema
  )

  with open(image_uri, 'rb') as f:
    raw_obj = base64.b64encode(f.read()).decode('utf-8')
    image1 = Part.from_data(data=base64.b64decode(raw_obj), mime_type="image/jpg")

  responses = model.generate_content(
      [prompt, image1],
      generation_config=generation_config,
      stream=False,
  )

  return responses.text

### Identification Step - To get the right keyword in Contents

In [15]:
import json

prompt = """

    **Instruction**
    - Identify the image. Show at most 3 possible identification.
    - Be very specific. Include location/brand/name if possible. Avoid adding description.
    - If identification is not possible, return an empty array
    """

image_uri = "./gd.jpg"

# Response schema can control the output of generation from Gemini.
response_schema = {
                "type": "ARRAY",
                "items": {"type": "STRING",},
            }

detection1 = detect_image(prompt, image_uri, response_schema)
print(detection1)

["G-Dragon", "Kwon Ji-yong", "Big Bang"]


### Categorizing Step - To get categirized value for supporting Search

In [16]:
prompt_transcript = """
    **Category List**
    FOOD: includes consumables that can be purchased from a store, such as fruits, vegetables, snacks
    DISH: includes consumables that can be ordered from a restaurant
    PUBLIC_LOCATION: includes physical locations that do not require ticket, such as park, city landmarks
    PRIVATE_LOCATION: includes physical locations that require ticket/booking/reservation, such as restaurant, museum, theme park
    HUMAN: includes historical figures, public figures, or a human
    CLOTHING: includes shirts, pants, shoes, anything a human can wear
    OTHERS: includes everything that does not belong to the defined categories
    **Instruction**
    - Map the image to the closest category. Only map to one category at most
    - Go straight to the category name. No description, no reasoning.
    """


# Response schema can control the output of generation from Gemini.
response_schema = {
                "type": "ARRAY",
                "items": {
                    "type": "STRING",
                },
            }

detection2 = detect_image(prompt_transcript, image_uri, response_schema)
print(detection2)

["HUMAN"]


In [17]:
prompt = f"""
  당신은 주어진 참고정보를 바탕으로 정보를 자세하게 정리해주는 AI 어시스턴트 입니다.
  1. 인물에 초점을 두고 정리해주세요.
  2. 정리는 가급적 bullet point로 자세하게 정리해주세요.
  3. 한국어로 답변해주세요.

  참고정보 : {detection1} - {detection2}

"""
response = search(prompt)
print_grounding_response(response)

## 권지용 (G-Dragon) 정보 정리 [1]


**개인 정보**

* **본명:** 권지용 (Kwon Ji-yong) [1]

* **출생:** 1988년 8월 18일, 서울, 대한민국 [1]

* **별명:** G-Dragon (지드래곤), GD [1]

* **직업:** 래퍼, 가수, 작곡가, 프로듀서, 사업가, 패션 디자이너 [1]

* **활동 기간:** 1994년 ~ 현재 [1]

* **학력:** 세종대학교 (석사)

**음악 경력**

* **장르:** 힙합, K-pop, 댄스 팝, R&B, 트랩 [1]

* **악기:** 보컬
* **소속사:** YG 엔터테인먼트, Galaxy Corp. [1]
 Empire
* **그룹:** 빅뱅 (Big Bang), GD & TOP, GD X 태양 [1]

* **과거 소속:** 리틀 루라, YG 패밀리 [1]


**주요 활동**

* **2006년:** 빅뱅으로 데뷔 [1]

* **2009년:** 첫 솔로 앨범 "Heartbreaker" 발매, 앨범과 타이틀곡 모두 큰 성공을 거두며 당시 한국 솔로 아티스트 최고 판매량 기록 [1]

* **2010년:** 빅뱅 멤버 T.O.P와 함께 "GD & TOP" 앨범 발매 [1]

* **2012년:** 첫 EP "One of a Kind" 발매, "One of a Kind", "Crayon", "That XX" 등 히트곡 탄생 [1]

* **2013년:** 첫 월드 투어 진행, 한국 솔로 아티스트 최초로 일본 돔 경기장 투어 [1]

* **2017년:** EP "Kwon Ji Yong" 발매, 타이틀곡 "Untitled, 2014" 차트 1위 기록 [1]

* **2017년:** 콘서트 투어 "Act III: M.O.T.T.E" 진행 [1]


**기타**

* "K-pop의 황제"로 불림 [1]

* 뛰어난 음악적 재능과 패션 감각으로 인정받음
* 다양한 분야에서 활발하게 활동하며 영향력을 행사하는 인물 [1]


**추가 정보**

* 권지용은 빅뱅의 리더로, 빅뱅은 세계적으로 가장 많이 팔린 보이 밴드 중 하나입니다. [1]

* 권지용은 솔로 아티스트로서도 큰 성공을 거두었으며, 그의 음악은 한국 음악 산업에 큰 영향을 미쳤습니다.
* 권지용은 패션 디자이너로서도 활동하며, 그의 패션 감각은 많은 사람들에게 영향을 주고 있습니다.

----
## Grounding Sources

**Web Search Queries:** ['Big Bang', 'G-Dragon', 'Kwon Ji-yong']

**Search Entry Point:**
 <style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webkit-tap-highlight-color: transparent;
}
.carousel {
  overflow: auto;
  scrollbar-width: none;
  white-space: nowrap;
  margin-right: -12px;
}
.headline {
  display: flex;
  margin-right: 4px;
}
.gradient-container {
  position: relative;
}
.gradient {
  position: absolute;
  transform: translate(3px, -9px);
  height: 36px;
  width: 9px;
}
@media (prefers-color-scheme: light) {
  .container {
    background-color: #fafafa;
    box-shadow: 0 0 0 1px #0000000f;
  }
  .headline-label {
    color: #1f1f1f;
  }
  .chip {
    background-color: #ffffff;
    border-color: #d2d2d2;
    color: #5e5e5e;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #f2f2f2;
  }
  .chip:focus {
    background-color: #f2f2f2;
  }
  .chip:active {
    background-color: #d8d8d8;
    border-color: #b6b6b6;
  }
  .logo-dark {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #fafafa 15%, #fafafa00 100%);
  }
}
@media (prefers-color-scheme: dark) {
  .container {
    background-color: #1f1f1f;
    box-shadow: 0 0 0 1px #ffffff26;
  }
  .headline-label {
    color: #fff;
  }
  .chip {
    background-color: #2c2c2c;
    border-color: #3c4043;
    color: #fff;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #353536;
  }
  .chip:focus {
    background-color: #353536;
  }
  .chip:active {
    background-color: #464849;
    border-color: #53575b;
  }
  .logo-light {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #1f1f1f 15%, #1f1f1f00 100%);
  }
}
</style>
<div class="container">
  <div class="headline">
    <svg class="logo-light" width="18" height="18" viewBox="9 9 35 35" fill="none" xmlns="http://www.w3.org/2000/svg">
      <path fill-rule="evenodd" clip-rule="evenodd" d="M42.8622 27.0064C42.8622 25.7839 42.7525 24.6084 42.5487 23.4799H26.3109V30.1568H35.5897C35.1821 32.3041 33.9596 34.1222 32.1258 35.3448V39.6864H37.7213C40.9814 36.677 42.8622 32.2571 42.8622 27.0064V27.0064Z" fill="#4285F4"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 43.8555C30.9659 43.8555 34.8687 42.3195 37.7213 39.6863L32.1258 35.3447C30.5898 36.3792 28.6306 37.0061 26.3109 37.0061C21.8282 37.0061 18.0195 33.9811 16.6559 29.906H10.9194V34.3573C13.7563 39.9841 19.5712 43.8555 26.3109 43.8555V43.8555Z" fill="#34A853"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M16.6559 29.8904C16.3111 28.8559 16.1074 27.7588 16.1074 26.6146C16.1074 25.4704 16.3111 24.3733 16.6559 23.3388V18.8875H10.9194C9.74388 21.2072 9.06992 23.8247 9.06992 26.6146C9.06992 29.4045 9.74388 32.022 10.9194 34.3417L15.3864 30.8621L16.6559 29.8904V29.8904Z" fill="#FBBC05"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 16.2386C28.85 16.2386 31.107 17.1164 32.9095 18.8091L37.8466 13.8719C34.853 11.082 30.9659 9.3736 26.3109 9.3736C19.5712 9.3736 13.7563 13.245 10.9194 18.8875L16.6559 23.3388C18.0195 19.2636 21.8282 16.2386 26.3109 16.2386V16.2386Z" fill="#EA4335"/>
    </svg>
    <svg class="logo-dark" width="18" height="18" viewBox="0 0 48 48" xmlns="http://www.w3.org/2000/svg">
      <circle cx="24" cy="23" fill="#FFF" r="22"/>
      <path d="M33.76 34.26c2.75-2.56 4.49-6.37 4.49-11.26 0-.89-.08-1.84-.29-3H24.01v5.99h8.03c-.4 2.02-1.5 3.56-3.07 4.56v.75l3.91 2.97h.88z" fill="#4285F4"/>
      <path d="M15.58 25.77A8.845 8.845 0 0 0 24 31.86c1.92 0 3.62-.46 4.97-1.31l4.79 3.71C31.14 36.7 27.65 38 24 38c-5.93 0-11.01-3.4-13.45-8.36l.17-1.01 4.06-2.85h.8z" fill="#34A853"/>
      <path d="M15.59 20.21a8.864 8.864 0 0 0 0 5.58l-5.03 3.86c-.98-2-1.53-4.25-1.53-6.64 0-2.39.55-4.64 1.53-6.64l1-.22 3.81 2.98.22 1.08z" fill="#FBBC05"/>
      <path d="M24 14.14c2.11 0 4.02.75 5.52 1.98l4.36-4.36C31.22 9.43 27.81 8 24 8c-5.93 0-11.01 3.4-13.45 8.36l5.03 3.85A8.86 8.86 0 0 1 24 14.14z" fill="#EA4335"/>
    </svg>
    <div class="gradient-container"><div class="gradient"></div></div>
  </div>
  <div class="carousel">
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfxSCv85uiMcDaSujii-_kLd3ZC5DmWQIdpetHrYIZu7WBSz4nsWUVivISGa-FNVqt4iHbSW-zaNNGmnKH-UOZfEY3i29SbipITeCWRNs_wIJVBTqWNmKXyqOOnutFKUZHxj-KxMy87uhN_n8rZxMuNEpoVUlDcfMq7M2iCojbOoybByZAlYyZz1buHU">Big Bang</a>
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfxJumVLxYo9Jm9sohUnE2DJXJeZopU2AYY5o43HNZpShA76_r9rbnIQodLoBajimuRemoDfJSnEchQFqnc4Hf5uIpDm4gRGUedYJ2GCcS1pt6Uhke_ZoxQoAaXIlgdArZnskiy27u7a_V3PI6STs6k-uD5xh_cODXyDZGZrSPzrKc6Y5wi3XiGMKk-x">G-Dragon</a>
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfxZLaQayC9_LQShIrj63Tw2YnfzX4BKE5RXvlucqnfWEQnciRRfcvEFKM2pTqNjoMBi5urlzxRKKFMLTTYbG21Q6JnXBP_COxq4hsvKZJIRGAv_J-u2NBSRuAReCLnzNO8kN0_yBy6sbKv_eLWisI6iwRfFSq2g6trX-6KigimJuy5pu08Z3YRzX9JsGfY8Yg==">Kwon Ji-yong</a>
  </div>
</div>

### Grounding Chunks
1. [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfzsmCGA8r5LbLJDqNNStTJw2WbeC9ha9XY_Xvs_BE3KU3VucoVO2oTvECuIFO2BwnFmqjnn6gOGPrLLHNldOH0kTKM5kGbmZ3sFu9ZPOlI_5UgBRLvwfqfdBWBeMP2XxyU1)


- https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfzsmCGA8r5LbLJDqNNStTJw2WbeC9ha9XY_Xvs_BE3KU3VucoVO2oTvECuIFO2BwnFmqjnn6gOGPrLLHNldOH0kTKM5kGbmZ3sFu9ZPOlI_5UgBRLvwfqfdBWBeMP2XxyU1

